In [ ]:
# Importing necessary libraries for ResNet traditional data augmentation
import os  
import re  
import cv2  
import random  
import numpy as np  
import pandas as pd  
import tensorflow as tf 

# Try to import tensorflow_addons, if not available, define alternative metrics
try:
    import tensorflow_addons as tfa
    HAS_TFA = True
except ImportError:
    print("Warning: tensorflow_addons not available. Using alternative F1 score implementation.")
    HAS_TFA = False
    
    # Define a simple F1 score metric as alternative
    class F1Score(tf.keras.metrics.Metric):
        def __init__(self, num_classes, average='weighted', name='f1', **kwargs):
            super(F1Score, self).__init__(name=name, **kwargs)
            self.num_classes = num_classes
            self.average = average
            self.precision = tf.keras.metrics.Precision()
            self.recall = tf.keras.metrics.Recall()
            
        def update_state(self, y_true, y_pred, sample_weight=None):
            y_pred = tf.argmax(y_pred, axis=1)
            y_true = tf.argmax(y_true, axis=1)
            self.precision.update_state(y_true, y_pred, sample_weight)
            self.recall.update_state(y_true, y_pred, sample_weight)
            
        def result(self):
            p = self.precision.result()
            r = self.recall.result()
            return 2 * (p * r) / (p + r + tf.keras.backend.epsilon())
            
        def reset_state(self):
            self.precision.reset_state()
            self.recall.reset_state()

# Setting the path to the directory containing preprocessed images
DATA_PATH = "/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images"

# Defining the target image size for resizing images
IMG_SIZE = 224

# Loading the dataset from a CSV file into a pandas DataFrame
data = pd.read_csv("/kaggle/input/ocular-disease-recognition-odir5k/full_df.csv")


In [ ]:
# Mapping of short class labels to their full descriptive names
class_short2full = {
    "G": "Glaucoma",  # Short label 'G' represents Glaucoma
    "C": "Cataract",  # Short label 'C' represents Cataract
    "A": "Age Related Macular Degeneration",  # Short label 'A' represents ARMD
    "H": "Hypertension",  # Short label 'H' represents Hypertension
    "M": "Myopia"  # Short label 'M' represents Myopia
}

# Mapping of short class labels to numerical indices for machine learning models
class_dict = {
    "G": 0,  # Glaucoma is assigned index 0
    "C": 1,  # Cataract is assigned index 1
    "A": 2,  # ARMD is assigned index 2
    "H": 3,  # Hypertension is assigned index 3
    "M": 4   # Myopia is assigned index 4
}


In [ ]:
# Data preprocessing and converting class labels
data["class"] = data["labels"].apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))

CLASSES = ["G", "C", "A", "H", "M"]


In [ ]:
# Create a dictionary mapping each class to a list of image filenames
dict_img_list = {
    class_: data.loc[data["class"] == class_]["filename"].values
    for class_ in class_short2full.keys()
}


In [ ]:
# Enhanced function to create a dataset by processing images from the given list
# Supports traditional augmentations (flipping, rotation, brightness, contrast) for ResNet training

def create_augmented_dataset(img_list, class_label, max_images, augment_factor=3):
    dataset = []  # Initialize an empty list to store processed images and their labels
    count = 0  # Counter to track the number of processed images

    # Loop through each image in the provided list
    for img in img_list:
        # Stop processing if the max_images limit is reached
        if max_images is not None and count >= max_images:
            break

        # Construct the full image path and read the image
        image_path = os.path.join(DATA_PATH, img)
        image = cv2.imread(image_path)

        # Skip if the image couldn't be loaded
        if image is None:
            continue

        # Convert the image to RGB format and resize to the target size
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

        # Add the original image
        dataset.append([np.array(image), class_label])
        count += 1

        # Apply traditional augmentations
        if augment_factor > 0:
            for _ in range(augment_factor):
                # Horizontal flip
                flipped_h = cv2.flip(image, 1)
                dataset.append([np.array(flipped_h), class_label])
                
                # Vertical flip
                flipped_v = cv2.flip(image, 0)
                dataset.append([np.array(flipped_v), class_label])
                
                # Rotation (90 degrees)
                rotated = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
                dataset.append([np.array(rotated), class_label])
                
                # Brightness adjustment
                brightened = cv2.convertScaleAbs(image, alpha=1.2, beta=20)
                dataset.append([np.array(brightened), class_label])
                
                # Contrast adjustment
                contrasted = cv2.convertScaleAbs(image, alpha=1.3, beta=0)
                dataset.append([np.array(contrasted), class_label])
                
                count += 5

    # Shuffle the dataset to randomize the order of images
    random.shuffle(dataset)

    return dataset


In [ ]:
# Dataset preparation for different ocular disease classes with traditional augmentations
# Initialize lists to store the datasets for each class

dataset_G = []  # Glaucoma dataset
dataset_C = []  # Cataract dataset  
dataset_A = []  # AMD dataset
dataset_H = []  # Hypertension dataset
dataset_M = []  # Myopia dataset

# Process each class with traditional augmentations
for i, class_ in enumerate(CLASSES):
    img_list = dict_img_list[class_]  # Get the list of images for the current class
    class_label = class_dict[class_]  # Get the class label
    
    print(f"Processing {class_short2full[class_]} with traditional augmentations...")
    
    if class_ == "G":
        dataset_G = create_augmented_dataset(img_list, class_label, 284, augment_factor=2)
    elif class_ == "C":
        dataset_C = create_augmented_dataset(img_list, class_label, 293, augment_factor=2)
    elif class_ == "A":
        dataset_A = create_augmented_dataset(img_list, class_label, 266, augment_factor=2)
    elif class_ == "H":
        dataset_H = create_augmented_dataset(img_list, class_label, 128, augment_factor=2)
    elif class_ == "M":
        dataset_M = create_augmented_dataset(img_list, class_label, 232, augment_factor=2)
    
    print(f"Completed {class_short2full[class_]} - Total samples: {len(eval(f'dataset_{class_}'))}")

# Print dataset sizes
print(f"\nDataset sizes after traditional augmentation:")
print(f"Glaucoma: {len(dataset_G)} samples")
print(f"Cataract: {len(dataset_C)} samples")
print(f"AMD: {len(dataset_A)} samples")
print(f"Hypertension: {len(dataset_H)} samples")
print(f"Myopia: {len(dataset_M)} samples")


In [ ]:
# Combine all augmented datasets
dataset = []
dataset += dataset_G + dataset_C + dataset_A + dataset_M + dataset_H

# Get the total number of samples in the combined dataset
print(f"Total augmented dataset size: {len(dataset)}")

# Shuffle the combined dataset
random.shuffle(dataset)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Parameters for data splitting
image_size = 224
num_classes = 5
train_ratio = 0.7
val_ratio = 0.15

# Preparing predictors and target variables
train_x = np.array([i[0] for i in dataset]).reshape(-1, image_size, image_size, 3)
train_y = np.array([i[1] for i in dataset])

# Calculating the number of images for each split
num_images = len(train_x)
num_train = int(num_images * train_ratio)
num_val = int(num_images * val_ratio)
num_test = num_images - num_train - num_val

# Splitting the dataset into train and remaining (validation + test)
x_train, x_remaining, y_train, y_remaining = train_test_split(train_x, train_y, train_size=num_train, random_state=42)

# Further splitting the remaining data into validation and test
x_val, x_test, y_val, y_test = train_test_split(x_remaining, y_remaining, test_size=num_test, random_state=42)

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

# Print the number of images in each split
print(f"Number of images - Train: {len(x_train)}, Validation: {len(x_val)}, Test: {len(x_test)}")


## ResNet50 Model Training with Traditional Augmentation


In [ ]:
# Define the image size
image_size = 224

# Import necessary layers from TensorFlow Keras
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D  
from tensorflow.keras.applications import ResNet50  
import tensorflow_addons as tfa

# Load the ResNet50 model pre-trained on ImageNet, excluding the top layer
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

# Set all ResNet50 layers as trainable
for layer in resnet.layers:
    layer.trainable = True

# Import the Sequential API for model creation
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense

# Initialize the Sequential model
model = Sequential()
# Add the ResNet50 model as a feature extractor (without the top layers)
model.add(resnet)

# Add Dropout layer to prevent overfitting (rate of 0.5)
model.add(Dropout(0.5))
# Add Global Average Pooling layer to reduce dimensionality of the output from ResNet50
model.add(GlobalAveragePooling2D())

# Flatten the pooled features for dense layer processing
model.add(Flatten())

# Add BatchNormalization to standardize activations and improve training speed
model.add(tf.keras.layers.BatchNormalization())

# Add a dense layer with 512 neurons and ReLU activation
model.add(Dense(512, activation="relu"))
# Add another dense layer with 256 neurons and ReLU activation
model.add(Dense(256, activation="relu"))
# Add a dense layer with 128 neurons and ReLU activation
model.add(Dense(128, activation="relu"))
# Add the final output layer with 5 neurons for 5 classes, using softmax for multi-class classification
model.add(Dense(5, activation="softmax"))

# Display the summary of the model architecture
model.summary()


In [ ]:
# Define the metrics to be tracked during training
METRICS = [
    tf.keras.metrics.AUC(name="auc"),  # Area under the ROC curve metric
    tf.keras.metrics.CategoricalAccuracy(name="acc"),  # Accuracy for categorical classification
    tfa.metrics.F1Score(num_classes=5, average="weighted", name="f1"),  # F1 score, weighted average across all classes
    tf.keras.metrics.AUC(name="prc", curve="PR"),  # Area under the Precision-Recall curve
]

# Compile the model with Adam optimizer, categorical crossentropy loss, and the specified metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Adam optimizer with a learning rate
    loss="categorical_crossentropy",  # Loss function for multi-class classification
    metrics=METRICS  # List of metrics to evaluate during training
)


In [ ]:
# Define the number of epochs for training
epochs = 50

# Import necessary callback modules from TensorFlow Keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Reduce learning rate on plateau callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    factor=0.5,  # Multiplies the learning rate by this factor when activated
    patience=15,  # Number of epochs to wait for improvement before reducing LR
    verbose=1,  # Print messages when learning rate is reduced
    min_delta=0.0001,  # Minimum change to qualify as an improvement
    cooldown=0,  # Number of epochs to wait before resuming normal learning rate
    min_lr=1e-7,  # Minimum learning rate, prevents LR from going below this value
)

# EarlyStopping callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=epochs // 5,  # Number of epochs without improvement before stopping
    restore_best_weights=True,  # Restore model weights from the epoch with the best performance
    verbose=1,  # Print messages when early stopping is triggered
)

# ModelCheckpoint callback
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("/kaggle/working/resnet_traditional_aug_model.h5", save_best_only=True)

# List of callbacks to be used during model training
callbacks = [checkpoint_cb, early_stopping_cb, reduce_lr]

# Train the model with callbacks
history = model.fit(x_train, y_train, batch_size=32, epochs=epochs, validation_data=(x_val, y_val), callbacks=callbacks)
print(history)


In [ ]:
# Save the augmented training data for future use
np.save('/kaggle/working/x_train_traditional_aug.npy', x_train)
np.save('/kaggle/working/y_train_traditional_aug.npy', y_train)

np.save('/kaggle/working/x_val_traditional_aug.npy', x_val)
np.save('/kaggle/working/y_val_traditional_aug.npy', y_val)

np.save('/kaggle/working/x_test_traditional_aug.npy', x_test)
np.save('/kaggle/working/y_test_traditional_aug.npy', y_test)

print("Traditional augmented datasets saved successfully!")
